In [1]:
import pickle

import tensorflow as tf
import pandas as pd
import numpy as np
from annoy import AnnoyIndex
import sys
import os
%matplotlib inline

In [3]:
import tensorflow_recommenders as tfrs
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf


In [4]:
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
from deep_recommender_embeddings.src.ares import request_asset_from_ares
from deep_recommender_embeddings.src.elasticsearch_utils import get_es_instance, get_data_from_es, print_item
from deep_recommender_embeddings.src.image_embeddings import generate_image_embeddings
from deep_recommender_embeddings.src.models import ItemSimilarityModel, UserItemModel
from deep_recommender_embeddings.src.plotting import plot_metric
from deep_recommender_embeddings.src.tf_utils import get_tf_lookup_table_for_property, get_tf_lookup_for_dict
from deep_recommender_embeddings.src.inference import get_dict_of_embeddings, build_annoy_index
from deep_recommender_embeddings.src.io import export_embeddings_to_file
from deep_recommender_embeddings.src.preprocessing import load_data, clean_data, filter_logs, get_pairs, \
    get_item_pairs_from_journeys, generate_model_recs, get_pairs_location

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


# Load content data

In [11]:
_es_index = 'news_20210305c'
_es_host = 'localhost'
_es_port = '9200'
es = get_es_instance(es_host=_es_host, es_port=_es_port)

# This is the imaginary date that recommendations will be generated on
prediction_time = "2020-11-02T00:00:00"

# Business rules usually prevent recs being returned that are older than 90 days so we will only retrieve items
# that were published within this time window
max_age_days = 90

In [12]:
features = ["combinedBodySummaryHeadline", "tagsText", "articleCategoryName"]
hits = get_data_from_es(es, features, 50000, 5000, prediction_time, max_age_days, _es_index)
unique_item_ids = [hit['sort'][0] for hit in hits]
print(f'Grabbed {len(unique_item_ids)} items from elasticsearch')

0
1
2
3
4
5
6
7
8
9
Grabbed 42732 items from elasticsearch


# Load user data

In [8]:
DATE_FIELD = 'event_start_datetime'
USER_FIELD = 'audience_id'

In [9]:
DATA_PATH = '/Users/mercef02/Projects/datasets_models/news_uk_logs_20210304/00*' #'/Users/mercef02/Projects/datasets/sfv_user_week_compact/*.csv'
logs = load_data(d_path=DATA_PATH, stop_after_n_files=1, date_field=DATE_FIELD)


In [10]:
logs = clean_data(logs, [USER_FIELD, DATE_FIELD, 'url', 'geo_city_site_visited'], '/news/')
min_date = pd.to_datetime('2021-03-04T00:00:00')
max_date = pd.to_datetime('2021-03-04T11:59:59')
min_mentions = 10

In [13]:
train_logs = filter_logs(logs, min_date, max_date, min_mentions, unique_item_ids, DATE_FIELD)

Original interactions: 1685668
Date filtered interactions: 670621
Minimum mention filtered interactions  : 653335
Interactions filtered for items existing in ES : 559809


In [14]:
train_logs.head()

,audience_id,event_start_datetime,geo_city_site_visited,uri
29,lpucBML/6soz1U/TcHk93BCzL/Ki+P+q6RxwKDWN4fg=,2021-03-04 02:52:31,Southwark,/news/technology-56239242
39,sPYIdNHgYJ98Gpw96yDcQgsFpeQfC0EJpPaSpVscw34=,2021-03-04 01:52:28,Manchester,/news/world-asia-55472446
158,3Lr5OzuexWD6m0bj08MikLbtjwGhj/Fc2gU3Z7tiSzY=,2021-03-04 00:35:18,Wakefield,/news/uk-56272104
159,G5pAYXT347wHjlC4ONQYVdsMAwSFtl1ysjRud0XE2n8=,2021-03-04 00:50:39,Omagh,/news/uk-56267807
160,rE3xy1mrmqGZ7P77YYs7y+e3/VqH+zu2Pp0Btu3Tp34=,2021-03-04 00:07:06,Stockton-on-tees,/news/business-56263581


In [98]:
np.random.permutation(train_pairs['candidate_item'].to_numpy())

array(['/news/uk-56278445', '/news/uk-56272104', '/news/health-55045639',
       ..., '/news/health-56271627',
       '/news/uk-england-northamptonshire-56274524', '/news/56238260'],
      dtype=object)

In [108]:
train_pairs_pos = train_logs[['geo_city_site_visited', 'uri']]
train_pairs_pos.columns = ['user_location', 'candidate_item']
train_pairs_pos.loc[:, 'user_rating'] = np.ones(shape=(len(train_pairs), 1))

train_pairs_neg = train_pairs_pos.copy()
train_pairs_neg.loc[:, 'candidate_item'] = np.random.permutation(train_pairs['candidate_item'].to_numpy())
train_pairs_neg.loc[:, 'user_rating'] = np.zeros(shape=(len(train_pairs), 1))

train_pairs = pd.concat([train_pairs_pos, train_pairs_neg], axis=0)



print(f"Total number of pairs: {len(train_pairs)}")
train_logs_tf = tf.data.Dataset.from_tensor_slices(dict(train_pairs))

/Users/mercef02/.local/share/virtualenvs/deep-recommender-embeddings-vIGdqmCw/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/mercef02/.local/share/virtualenvs/deep-recommender-embeddings-vIGdqmCw/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Total number of pairs: 1119618


In [110]:
len(train_pairs_pos)

559809

In [27]:
location_vocab=np.array(train_logs.geo_city_site_visited.unique())

In [47]:
train_pairs.head()

,user_location,candidate_item,user_rating
29,Southwark,/news/technology-56239242,1.0
39,Manchester,/news/world-asia-55472446,1.0
158,Wakefield,/news/uk-56272104,1.0
159,Omagh,/news/uk-56267807,1.0
160,Stockton-on-tees,/news/business-56263581,1.0


In [142]:
from tensorflow.python.keras.initializers.initializers_v2 import VarianceScaling
import uuid

class RankingModel(tf.keras.Model):

  def __init__(self, features=['location', 'item_id'], feature_dims=[32, 32], layer_sizes=[256, 64]):
    super().__init__()
    
    for i, feature in enumerate(features):
        
        if feature == 'location':
            # Compute embeddings for users.
            self.location_embeddings = tf.keras.Sequential([
              tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=location_vocab, mask_token=None),
              tf.keras.layers.Embedding(len(location_vocab) + 1, feature_dims[i])
            ])
        
        if feature == 'item_id':
            # Compute embeddings for items.
            self.item_embeddings = tf.keras.Sequential([
              tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=unique_item_ids, mask_token=None),
              tf.keras.layers.Embedding(len(unique_item_ids) + 1, feature_dims[i])
            ])
        
        if feature == 'user_id':
            # Compute embeddings for items.
            self.user_embeddings = tf.keras.Sequential([
              tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=unique_item_ids, mask_token=None),
              tf.keras.layers.Embedding(len(unique_item_ids) + 1, feature_dims[i])
            ])            

     # Then construct the layers.
    self.ratings = tf.keras.Sequential(name='sequential_deep')
    
    # Compute predictions.
    # Use the ReLU activation for all but the last layer.
    for i, layer_size in enumerate(layer_sizes):
        
        # Batch normalization after the first layer
        if i == 0:
             self.ratings.add(tf.keras.layers.BatchNormalization())
                
        self.ratings.add(tf.keras.layers.Dense(layer_size,
                                                    activation="relu",
                                                    kernel_initializer=VarianceScaling(),
                                                    bias_initializer=VarianceScaling(),
                                                    name=f"dense_layer_{i + 1}_{str(uuid.uuid4())}"))
    # Make rating predictions in the final layer.
    self.ratings.add(tf.keras.layers.Dense(1))


  def call(self, inputs):

    user_location, item_id = inputs

    location_embedding = self.location_embeddings(user_location)
    item_embedding = self.item_embeddings(item_id)

    return self.ratings(tf.concat([location_embedding, item_embedding], axis=1))



In [143]:
class Reranker(tfrs.models.Model):

  def __init__(self, features=['location', 'item_id'], feature_dims=[32, 32], layer_sizes=[256, 64]):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel(features=features, feature_dims=feature_dims, layer_sizes=layer_sizes)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    rating_predictions = self.ranking_model(
        (features["user_location"], features["candidate_item"]))

    # The task computes the loss and the metrics.
    return self.task(labels=features["user_rating"], predictions=rating_predictions)

In [144]:
tf.random.set_seed(42)
training_shuffled = train_logs_tf.shuffle(buffer_size=100_000, seed=42, reshuffle_each_iteration=False)
train_pc = 0.98
test_pc = 0.02
news_train = training_shuffled.take(np.floor(len(training_shuffled)*train_pc))
news_val = training_shuffled.skip(np.floor(len(training_shuffled)*train_pc)).take(np.floor(len(training_shuffled)*test_pc))

print(f'Number of train pairs: {len(news_train)}')
print(f'Number of test pairs: {len(news_val)}')

Number of train pairs: 1097225
Number of test pairs: 22392


In [148]:
train_batch = 8192
test_batch = 4096
lr = 0.01
epochs=10
cached_train = news_train.shuffle(100_000).batch(train_batch).cache()
cached_test = news_val.batch(test_batch).cache()
model = Reranker(features=['location', 'item_id'], feature_dims=[64, 64], layer_sizes=[64, 32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=lr))
model.fit(cached_train, epochs=epochs)

Epoch 1/10
134/134 [==============================] - 11s 76ms/step - root_mean_squared_error: 0.3305 - loss: 0.1085 - regularization_loss: 0.0000e+00 - total_loss: 0.1085
Epoch 2/10
134/134 [==============================] - 4s 28ms/step - root_mean_squared_error: 0.3889 - loss: 0.1502 - regularization_loss: 0.0000e+00 - total_loss: 0.1502
Epoch 3/10
134/134 [==============================] - 4s 29ms/step - root_mean_squared_error: 0.4168 - loss: 0.1726 - regularization_loss: 0.0000e+00 - total_loss: 0.1726
Epoch 4/10
134/134 [==============================] - 4s 26ms/step - root_mean_squared_error: 0.4366 - loss: 0.1894 - regularization_loss: 0.0000e+00 - total_loss: 0.1894
Epoch 5/10
134/134 [==============================] - 4s 28ms/step - root_mean_squared_error: 0.4510 - loss: 0.2021 - regularization_loss: 0.0000e+00 - total_loss: 0.2021
Epoch 6/10
134/134 [==============================] - 4s 30ms/step - root_mean_squared_error: 0.4614 - loss: 0.2116 - regularization_loss: 0.000

In [113]:
model.evaluate(cached_test, return_dict=True)

6/6 [==============================] - 5s 29ms/step - root_mean_squared_error: 0.1023 - loss: 0.0107 - regularization_loss: 0.0000e+00 - total_loss: 0.0107


{'root_mean_squared_error': 0.10227575898170471,
 'loss': 0.011401545256376266,
 'regularization_loss': 0,
 'total_loss': 0.011401545256376266}

In [90]:
def get_top_n_ranked(model, query_location, candidate_list, n):
    query_input = (np.repeat(query_location, len(candidate_list)), candidate_list)
    scored_candidates = model.ranking_model(query_input)
    results = pd.DataFrame.from_dict({'uri': candidate_list, "scores": scored_candidates.numpy().flatten()})
    sorted_results = results.sort_values('scores', ascending=False)
    print(sorted_results.head(n))
    return sorted_results


In [119]:
candidate_list = np.array(unique_item_ids)
query_location = 'Newcastle'
scored_results = get_top_n_ranked(model, query_location, candidate_list, 30)


                                                uri    scores
14341                       /news/newsbeat-33148029  0.096374
2779              /news/entertainment-arts-55978659  0.096130
16584                       /news/newsbeat-35584995  0.093983
9867                        /news/newsbeat-24691515  0.093910
35955                       /news/uk-wales-55332202  0.093799
19389                       /news/newsbeat-39154348  0.093777
34485  /news/uk-scotland-highlands-islands-55015585  0.093741
21920                     /news/technology-55684595  0.093711
23348                             /news/uk-56005071  0.093678
35190    /news/uk-scotland-tayside-central-55676707  0.093625
36740                       /news/uk-wales-56118730  0.093608
41492            /news/world-latin-america-56112386  0.093585
17309                       /news/newsbeat-36276060  0.093491
39681                   /news/world-europe-39140791  0.093455
19863                       /news/newsbeat-39951532  0.093379
5360    

In [83]:
results = pd.DataFrame.from_dict({'uri': candidate_list, "scores": scored_candidates.numpy().flatten()})


In [84]:
results = 

,uri,scores
0,/news/10205571,0.998301
1,/news/12211968,0.992364
2,/news/14295731,1.003392
3,/news/20039682,0.986870
4,/news/29881189,1.012518


In [88]:
results.sort_values('scores', ascending=False)

,uri,scores
38519,/news/world-asia-55368524,1.045556
35453,/news/uk-wales-53565399,1.043822
21728,/news/technology-54974813,1.040119
20869,/news/newsbeat-42026165,1.038186
1150,/news/business-55009742,1.038104
...,...,...
41793,/news/world-us-canada-24590082,0.957420
7887,/news/newsbeat-19341268,0.957281
23285,/news/uk-55890224,0.955544
6726,/news/newsbeat-15886812,0.955402
